In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow import keras
import pyttsx3

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_styled_landmarks(image,results):
    mp_drwaing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drwaing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drwaing.DrawingSpec(color=(80, 256, 121), thickness=1,  circle_radius=1)
                              )
    # Draw Face Connections
    mp_drwaing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drwaing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drwaing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)

                             # mp_drwaing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                             # mp_drwaing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                              )  # Draw Pose Connections
    mp_drwaing.draw_landmarks(image, results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drwaing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drwaing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                              )  # Draw Left Hand Connections
    mp_drwaing.draw_landmarks(image, results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drwaing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_drwaing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                              )  # Draw Right Hand Connections


In [6]:
mp_holistic = mp.solutions.holistic   #Holistoc model
mp_drwaing = mp.solutions.drawing_utils  #drwaing utilies

In [7]:
# ! pip install graphviz
# !pip install pydot
import graphviz
import pydot
import tensorflow as tf
model = keras.models.load_model('action2.h5')
# dot_img_file = 'model_1.png'
# tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)


In [8]:
# initialize Text-to-speech engine
engine = pyttsx3.init()

In [10]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [11]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Add','Agree','Approve','Argue','Arrive','Attack','Bite','Boil','Bond','Call','Cancel','Confrim','Delay','Destroy', 'Die','Draw','Drink','Dance','Deal','Dress', 'Explain','Easy','Fan','Fall','Fight','Fix','Find','Fly','Gather','Hug','Hello' , 'howAreYou' ,'Help','Hit','Hope','Injured','I','Irritate','Insult' ,'Kiss','Knock','Know','Learn','Laugh','Live','Like','Listen','Lie','Love','Make','Match','Need','Now', 'Power','Stop','Thanks','Try','Want','You'])
#'Hello' ,'Thanks', 'howAreYou', 'Fan' ,'Want','Add','Agree'
#'Insult','Love','Now','Try','Hit','Hope','Injured','Help','Easy'
#'Insult','Love','Now','Try','Hit','Hope','Injured','Help','Easy'
#'cancel','confrim','delay','Destroy', 'Die','draw',Drink,'Dance','Fall'
#'Dress', 'Power','Explain','Fight','Fix','Find','Fly''Gather','Irritate'
#'Hug','Kiss','Knock','Know','Learn','Laugh'
#'Live','Like','Listen','Lie','Make','Match','Need','Stop'
# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [13]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
        #3. Viz logic
#             if res[np.argmax(res)] > threshold:
            if np.unique(predictions[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
#                 engine.say(sentence)
#                 # play the speech
#                 engine.runAndWait()
                sentence = sentence[-5:]

#             # Viz probabilities
#             image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        
        # convert this text to speech
#         text = "Python is a great programming language"
#         engine.say(sentence)
#         # play the speech
#         engine.runAndWait()



        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Draw
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Draw
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Draw
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Draw
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Die
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Die
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Die
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Die
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Die
<class 'mediapipe.python.solution_base.SolutionOutputs'>


<class 'mediapipe.python.solution_base.SolutionOutputs'>
Kiss
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Cancel
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Irritate
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Cancel
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Arrive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Arrive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Arrive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Arrive
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Destroy
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Try
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Live
<class 'mediapipe.python.solution_base.S

In [14]:
# convert this text to speech
#text = "Python is a great programming language"
engine.say(sentence)
# play the speech
engine.runAndWait()

In [15]:
print(sentence)

['Fly', 'Fly', 'Confrim', 'Die', 'Approve']
